# Siberia March-May temperature anomalies
Seasonal average temperatures broken records were broken across Siberia in March-May 2020. Here we plot the region where ERA5 records were broken.

In this first section, we load required packages and modules

In [ ]:
##This is so variables get printed within jupyter
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
##import packages

import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cartopy
import cartopy.crs as ccrs
import matplotlib.ticker as mticker
from shapely import geometry


#for rank calculation
import bottleneck

In [ ]:
## this is to load our own function to retrieve ERA5,
## which is located in ../../src/CDSretrieve.py
import os
import sys

sys.path.insert(0, os.path.abspath('../../'))
os.chdir(os.path.abspath('../../'))
os.getcwd() #print the working directory

In [ ]:
##And here we load the module
import src.cdsretrieve as retrieve
import src.preprocess as preprocess

## Download ERA5

This section describes the retrieval of ERA5.
We retrieve netcdf files of global monthly 2m temperature and 2m dewpoint temperature for each year over 1979-2020. 
The data were retrieved in August 2020.

In [ ]:
retrieve.retrieve_ERA5(variables = ['2m_temperature','2m_dewpoint_temperature'], folder = '../Siberia_example/')

We load all files with xarray `open_mfdataset`. The latest 3 months in this dataset are made available through ERA5T, which might be slightly different to ERA5.
In the downloaded file, an extra dimenions 'expver' indicates which data is ERA5 (`expver = 1`) and which is ERA5T (`expver = 5`). 
After retrieving and loading, I combine both ERA5 and ERA5T to create a dataset that runs until August 2020.


In [ ]:
ERA5 = xr.open_mfdataset('../Siberia_example/ERA5_????.nc',combine='by_coords') ## open the data
ERA5

In [ ]:
ERA5_combine =ERA5.sel(expver=1).combine_first(ERA5.sel(expver=5))
ERA5_combine.load()

In [ ]:
month_length = ERA5_combine.time.dt.days_in_month
ERA5_seasonal = ((ERA5_combine * month_length).resample(time='QS-DEC').sum() / 
          month_length.resample(time='QS-DEC').sum())
ERA5_seasonal

## Calculating the rank

We want to show for each month whether the recorded monthly average temperature for 2020 is the highest since 1979 (or second highest, etc.). 

We first select only January months.

In [ ]:
ERA5_MAM = ERA5_seasonal.sel(time=ERA5_seasonal['time.month'] == 3) ## Select only for MAM

Then we calculate the rank of January average temperatures over the years. We rename the variable 't2m' into 'Temperature rank'. 

In [ ]:
ERA5_MAM_rank = ERA5_MAM['t2m'].rank(dim = 'time')
ERA5_MAM_rank = ERA5_MAM_rank.rename('Temperature rank')

We now have calculated the rank in increasing order, i.e. the highest values has the highest rank. 
However, we want to show the highest rank being number 1, the second highest being number 2. Therefore, we invert the ranks 
and then we select the inverted rank of January 2020 average temperature within the January average temperatures of the other years. If January 2020 average temperature would be highest on record, the inverted rank will be 1. Second highest will be 2.

In [ ]:
ERA5_MAM_rank_inverted = (len(ERA5_MAM_rank.time) - ERA5_MAM_rank + 1).sel(time='2020')
ERA5_MAM_rank_inverted

## Plotting

We define a function to plot the data on a global map:

In [ ]:
def Global_plot(ERA5_i_rank_inverted):
    fig, ax = plt.subplots(frameon=False, figsize=(90 / 25.4, 60 / 25.4))
    ax = plt.axes(projection=ccrs.Robinson())
    ERA5_i_rank_inverted.plot(
        ax=ax,
        transform=ccrs.PlateCarree(),
        levels=[1, 2, 3, 4, 5],
        extend='both',
        colors=plt.cm.Reds_r)

    ax.add_feature(cartopy.feature.BORDERS, linestyle=':')
    ax.coastlines(
        resolution='110m')  #Currently can be one of “110m”, “50m”, and “10m”.
    gl = ax.gridlines(crs=ccrs.PlateCarree(),
                      draw_labels=True,
                      linewidth=1,
                      color='gray',
                      alpha=0.5,
                      linestyle='--')
#     gl.top_labels = False
#     gl.right_labels = False
    geom = geometry.box(minx=65, # the domain we select
                        maxx=120,
                        miny=50,
                        maxy=70)
    ax.add_geometries([geom],
                      crs=cartopy.crs.PlateCarree(),
                      linewidth=3,
                      facecolor='none',
                      edgecolor='black')

And plot!

In [ ]:
### Set extent and projection
plt.rcParams["font.family"] = "sans-serif" ##change font
plt.rcParams['font.size'] = 10  ## change font size
plt.rcParams['svg.fonttype'] = 'none' ## so inkscape recognized texts in svg file
plt.rcParams['pdf.fonttype'] = 42 ## so illustrator can recognize text

In [ ]:
Global_plot(ERA5_MAM_rank_inverted)
# plt.savefig('graphs/Siberia_temprank.pdf')

And zoom in for Siberia. We define a new plot:

In [ ]:
def Siberia_plot(ERA5_i_rank_inverted):
    fig, ax = plt.subplots(figsize=(9, 4.5))
    ax = plt.axes(projection=ccrs.PlateCarree(central_longitude=50.0))
    ERA5_i_rank_inverted.plot(
        ax=ax,
        transform=ccrs.PlateCarree(),
        levels=[1, 2, 3, 4, 5],
        extend='both',
        colors=plt.cm.Reds_r)

    ax.add_feature(cartopy.feature.BORDERS, linestyle=':')
    ax.coastlines(resolution='50m')
    gl = ax.gridlines(crs=ccrs.PlateCarree(),
                      draw_labels=True,
                      linewidth=1,
                      color='gray',
                      alpha=0.5,
                      linestyle='--')
    gl.top_labels = False
    gl.right_labels = False
    
    geom = geometry.box(minx=65, # the domain we select
                        maxx=120,
                        miny=50,
                        maxy=70)
    ax.add_geometries([geom],
                      crs=cartopy.crs.PlateCarree(),
                      linewidth=3,
                      facecolor='none',
                      edgecolor='black')

In [ ]:
Siberia_plot(ERA5_MAM_rank_inverted.sel(longitude = slice(40,140), latitude = slice(80,40)))
# plt.savefig('graphs/Siberia_temprank_zoomed.pdf')

### plot the timeseries


In [ ]:
ERA5_anomaly = ERA5_MAM['t2m'] - ERA5_MAM['t2m'].sel(time=slice('1979','2010')).mean('time')
ERA5_anomaly.attrs = {
    'long_name': 'August temperature anomaly',
    'units': 'C'
}
ERA5_sd_anomaly = ERA5_anomaly / ERA5_MAM['t2m'].sel(time=slice('1979','2010')).std('time')
ERA5_sd_anomaly.attrs = {
    'long_name': 'August temperature standardized anomaly',
    'units': '-'
}

In [ ]:
area_weights_ERA = np.cos(np.deg2rad(ERA5_sd_anomaly.latitude))

ERA5_Siberia_events = (
    ERA5_sd_anomaly.sel(  # Select 2 metre temperature
        latitude=slice(70, 50),        # Select the latitudes
        longitude=slice(65, 120)).    # Select the longitude
    weighted(area_weights_ERA).       # weights 
    mean(['longitude', 'latitude']))  # Take the average

In [ ]:
plt.figure(frameon=False, figsize=(90 / 25.4, 60 / 25.4))
ERA5_Siberia_events.plot()
plt.ylabel('MAM standardized anomaly')
# plt.savefig('graphs/Siberia_anomaly_timeseries.pdf', bbox_layout = 'tight')